In [1]:
import pandas as pd
import numpy as np 
import sklearn.cluster
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from numpy.linalg import lstsq
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction import text 
import re
%matplotlib inline

In [2]:
def remove_html_tags(data):
    p = re.compile(r'<[^<]*?>')
    return p.sub('', data)
     

In [4]:
hilary = pd.read_csv('data/hilary_scores.csv')
hilary_pos = hilary[hilary['Sentiment_b']==1]
hilary_neg = hilary[hilary['Sentiment_b']==0]

vectorizer = TfidfVectorizer(stop_words='english', max_features = 5000)
list_ = [hilary_pos, hilary_neg]

for i in list_:
    print("##################################################")
    V = vectorizer.fit_transform(i['Comment'].values).toarray()
    features = vectorizer.get_feature_names()
    nmf = NMF(n_components=10).fit(V)

    for topic_idx, topic in enumerate(nmf.components_, 1):
        print("Topic #%d:" % topic_idx)
        print(" ".join([features[i]
                        for i in topic.argsort()[:-10 - 1:-1]]))


##################################################
Topic #1:
bernie sanders support socialist win media candidate senator campaign vote
Topic #2:
joe biden run man vp love legacy warren vice beau
Topic #3:
iran deal war nuclear israel obama agreement sanctions world bomb
Topic #4:
trump donald gop mr fox debate bush fiorina media candidates
Topic #5:
college government money pay education students tax free tuition wage
Topic #6:
hillary clinton mrs email campaign server bush woman democrats emails
Topic #7:
republican party vote democratic candidate candidates democrats republicans election win
Topic #8:
times nyt news story coverage new york ny article readers
Topic #9:
president obama vice mr running woman states united biden office
Topic #10:
people like don just time think know good want really
##################################################
Topic #1:
bernie sanders hillary campaign candidate supporters socialist angry article media
Topic #2:
times story news nyt coverage new yo

In [10]:
nmf.transform(V).shape

(6607, 10)

In [18]:
matrix = nmf.transform(V)

In [19]:
matrix.argmax(axis=0)

array([2465, 2312, 3752, 2188, 4705, 5233, 3062, 4114, 1477, 3236])

In [22]:
sanders = pd.read_csv('data/sanders_scores.csv')
sanders[pd.isnull(sanders['Comment'])] = ""

sanders_pos = sanders[sanders['Sentiment_b']==1]
sanders_neg = sanders[sanders['Sentiment_b']==0]

vectorizer = TfidfVectorizer(stop_words='english', max_features = 5000)

list_ = [sanders_pos, sanders_neg]

for i in list_:
    print("##################################################")
    
    V = vectorizer.fit_transform(i['Comment'].values).toarray()
    features = vectorizer.get_feature_names()
    nmf = NMF(n_components=10).fit(V)

    for topic_idx, topic in enumerate(nmf.components_, 1):
        print("Topic #%d:" % topic_idx)
        print(" ".join([features[i]
                        for i in topic.argsort()[:-10 - 1:-1]]))

##################################################
Topic #1:
hillary clinton candidate campaign mrs bush warren issues ms jeb
Topic #2:
bernie love sanders need vermont candidate run em like man
Topic #3:
people just like don think know time want country really
Topic #4:
vote got ll primary don election voting voted republican win
Topic #5:
sanders senator thank warren mr sen president 2016 socialist ticket
Topic #6:
class middle working poor income rich americans jobs term economic
Topic #7:
president obama legacy republicans barack 2012 deal did congress bush
Topic #8:
party republican democratic democrats right left republicans candidate candidates election
Topic #9:
tax cuts taxes pay social rich security income bush republicans
Topic #10:
government trade good american money jobs food world free public
##################################################
Topic #1:
people government american common country nation america believe poor leaders
Topic #2:
hillary clinton left woman bush 

In [24]:
biden = pd.read_csv('data/biden_scores.csv')
vectorizer = TfidfVectorizer(stop_words='english', max_features = 5000)

biden_pos = biden[biden['Sentiment_b']==1]
biden_neg = biden[biden['Sentiment_b']==0]

list_ = [biden_pos, biden_neg]

for i in list_:
    print("##################################################")
    V = vectorizer.fit_transform(i['Comment'].values).toarray()
    features = vectorizer.get_feature_names()
    nmf = NMF(n_components=10).fit(V)

    for topic_idx, topic in enumerate(nmf.components_, 1):
        print("Topic #%d:" % topic_idx)
        print(" ".join([features[i]
                        for i in topic.argsort()[:-10 - 1:-1]]))

##################################################
Topic #1:
bernie sanders support socialist win media candidate senator campaign vote
Topic #2:
joe biden run man vp love legacy warren vice beau
Topic #3:
iran deal war nuclear israel obama agreement sanctions world bomb
Topic #4:
trump donald gop mr fox debate bush fiorina media candidates
Topic #5:
college government money pay education students tax free tuition wage
Topic #6:
hillary clinton mrs email campaign server bush woman democrats emails
Topic #7:
republican party vote democratic candidate candidates democrats republicans election win
Topic #8:
times nyt news story coverage new york ny article readers
Topic #9:
president obama vice mr running woman states united biden office
Topic #10:
people like don just time think know good want really
##################################################
Topic #1:
bernie sanders hillary campaign candidate supporters socialist angry article media
Topic #2:
times story news nyt coverage new yo

In [20]:
df_states = pd.read_csv("data/df_states_truncated.csv")
len(df_states)

71997

In [21]:
trump = pd.read_csv('data/trump_scores.csv')
pd.isnull(trump['Comment']).sum()
#sanders[pd.isnull(sanders['Comment'])] = ""

trump_pos = trump[trump['Sentiment_b']==1]
trump_neg = trump[trump['Sentiment_b']==0]

vectorizer = TfidfVectorizer(stop_words='english', max_features = 5000)

list_ = [trump_pos, trump_neg]

for i in list_:
    print("##################################################")

    V = vectorizer.fit_transform(i['Comment'].values).toarray() 
    features = vectorizer.get_feature_names()
    nmf = NMF(n_components=10).fit(V)

    for topic_idx, topic in enumerate(nmf.components_, 1):
        print("Topic #%d:" % topic_idx)
        print(" ".join([features[i]
             for i in topic.argsort()[:-10 - 1:-1]]))

##################################################
Topic #1:
trump supporters support media says campaign polls money voters 2016
Topic #2:
people country american america immigration world americans immigrants political government
Topic #3:
bush jeb brother safe george kept clinton smarter family iraq
Topic #4:
sanders bernie hillary clinton brooks biden experience times democratic candidate
Topic #5:
mr brooks ramos bruni column thank blow right ms press
Topic #6:
debate candidates media news cnn fox time questions issues reality
Topic #7:
gop candidate candidates nomination women win run base election voters
Topic #8:
donald maureen great dowd column love man know thank new
Topic #9:
like president think just don really say know good want
Topic #10:
party republican republicans democratic candidates democrats candidate democrat tea voters
##################################################
Topic #1:
republican party base republicans candidates candidate voters democrats vote election

In [25]:
bush = pd.read_csv('data/bush_scores.csv')
vectorizer = TfidfVectorizer(stop_words='english', max_features = 5000)

bush_pos = bush[bush['Sentiment_b']==1]
bush_neg = bush[bush['Sentiment_b']==0]

list_ = [bush_pos, bush_neg]

for i in list_:
    print("##################################################")
                                     
    V = vectorizer.fit_transform(i['Comment'].values).toarray()
    features = vectorizer.get_feature_names()
    nmf = NMF(n_components=10).fit(V)

    for topic_idx, topic in enumerate(nmf.components_, 1):
        print("Topic #%d:" % topic_idx)
        print(" ".join([features[i]
                        for i in topic.argsort()[:-10 - 1:-1]]))

##################################################
Topic #1:
trump donald mr mccain media president america says said polls
Topic #2:
work tax hours jobs pay taxes working workers time income
Topic #3:
bush jeb brother george family smart florida smarter iraq campaign
Topic #4:
hillary president clinton biden joe run obama vote dowd win
Topic #5:
sanders bernie times coverage nyt media campaign candidate candidates clinton
Topic #6:
republican party republicans candidates candidate democratic democrats vote voters presidential
Topic #7:
people country american america immigrants immigration right americans want illegal
Topic #8:
iran deal israel nuclear war obama agreement sanctions world weapons
Topic #9:
like just don think good really know time say colbert
Topic #10:
gop candidates kasich win voters election candidate primary nomination walker
##################################################
Topic #1:
trump donald mr media mccain candidates comments american attention way
Topic #2

In [27]:
carson = pd.read_csv('data/carson_scores.csv')
carson[pd.isnull(carson['Comment'])] = ""                         

carson_pos = carson[carson['Sentiment_b']==1]
carson_neg = carson[carson['Sentiment_b']==0]

vectorizer = TfidfVectorizer(stop_words='english', max_features = 5000)

list_ = [carson_pos, carson_neg]

for i in list_:
    print("##################################################")

    V = vectorizer.fit_transform(i['Comment'].values).toarray() 
    features = vectorizer.get_feature_names()
    nmf = NMF(n_components=10).fit(V)

    for topic_idx, topic in enumerate(nmf.components_, 1):
        print("Topic #%d:" % topic_idx)
        print(" ".join([features[i]
                        for i in topic.argsort()[:-10 - 1:-1]]))

##################################################
Topic #1:
people don just think country white want like government need
Topic #2:
trump donald mr like supporters america support polls immigration say
Topic #3:
party republican republicans vote democratic candidates democrats voters tea election
Topic #4:
bush jeb brother florida safe george win kept election 11
Topic #5:
president hillary sanders bernie clinton mr brooks biden obama candidate
Topic #6:
debate fox candidates news questions debates night media did watch
Topic #7:
carson dr ben black doctor neurosurgeon obama surgeon care medical
Topic #8:
gop clown candidates car candidate just presidential voters 2016 field
Topic #9:
gail welcome missed collins great column thank glad good ve
Topic #10:
walker rubio like cruz paul fiorina rand kasich scott good
##################################################
Topic #1:
party republican republicans base gop candidates vote voters democratic presidential
Topic #2:
black lives matter 